<a href="https://colab.research.google.com/github/Rudra-prasad-tarai/CreditRiskOptimisation/blob/main/Credit_Risk_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
!git clone https://github.com/Rudra-prasad-tarai/CreditRiskOptimisation
!pip install odfpy

fatal: destination path 'CreditRiskOptimisation' already exists and is not an empty directory.


In [104]:
import pandas as pd
import numpy  as np
from tqdm import tqdm
import re
import datetime as dt

In [105]:
transition_matrix = pd.read_excel(
    '/content/CreditRiskOptimisation/data/CrisilTransitionMatrix.ods',
    engine='odf',
    skipfooter=2,          # Excludes last 2 rows
    index_col=0            # Uses 1st column as row labels (index)
)

In [106]:
bonds = pd.read_csv('/content/CreditRiskOptimisation/data/MW-Bonds-on-CM-18-Apr-2025.csv')


In [107]:
bonds.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)',
       'VALUE \n (₹ Crores)', ' \n', 'CREDIT RATING \n', 'MATURITY DATE \n'],
      dtype='object')

In [108]:
bonds.isnull().sum()

,0
SYMBOL \n,0
SERIES \n,15
BOND TYPE \n,118
COUPON RATE \n,0
FACE VALUE \n,0
LTP \n,0
%CHNG \n,0
VOLUME \n(Shares),0
VALUE \n (₹ Crores),0
\n,398


In [109]:
bonds['VOLUME \n(Shares)'].unique()
bonds['FACE VALUE \n'].unique()

array(['1,000.00', '5,000.00', '-', '300.00', '400.00'], dtype=object)

In [110]:
# Deleting those bonds which does not have crisil rating
del bonds[bonds.columns[8]]
bonds.dropna(subset = ['CREDIT RATING \n'],inplace=True) #removed those whoo does not having crisil rating
print(bonds.columns)

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)', ' \n',
       'CREDIT RATING \n', 'MATURITY DATE \n'],
      dtype='object')


In [111]:
bonds['VOLUME \n(Shares)']

,VOLUME \n(Shares)
0,"5,678"
1,"4,469"
2,"4,008"
3,"2,515"
4,"2,760"
...,...
388,-
389,-
390,-
391,-


In [112]:


# Extract ONLY CRISIL ratings (case-insensitive, handles variations)
bonds['CREDIT RATING \n'] = bonds['CREDIT RATING \n'].str.extract(r'(CRISIL\s*[A-Za-z+-]+\s*[A-Za-z]*)', flags=re.IGNORECASE)

# Set non-CRISIL entries to NaN
bonds.loc[~bonds['CREDIT RATING \n' ].str.contains('CRISIL', na=False, case=False), 'CREDIT RATING'] = pd.NA

# Preprocessing
bonds['VOLUME \n(Shares)'] = bonds['VOLUME \n(Shares)'].str.replace('-', '0')
bonds['VOLUME \n(Shares)'] = bonds['VOLUME \n(Shares)'].str.replace(',', '').astype(int)  # or `float`
bonds['FACE VALUE \n'] = bonds['FACE VALUE \n'].str.replace('-', '0')
bonds['FACE VALUE \n'] = bonds['FACE VALUE \n'].str.replace(',', '').astype(float)
bonds['COUPON RATE \n'] = bonds['COUPON RATE \n'].str.replace('-', '0')
bonds['COUPON RATE \n'] = bonds['COUPON RATE \n'].str.replace(',', '').astype(float)   # or `float`



In [113]:
bonds.columns

Index(['SYMBOL \n', 'SERIES \n', 'BOND TYPE \n', 'COUPON RATE \n',
       'FACE VALUE \n', 'LTP \n', '%CHNG \n', 'VOLUME \n(Shares)', ' \n',
       'CREDIT RATING \n', 'MATURITY DATE \n', 'CREDIT RATING'],
      dtype='object')

In [114]:
bonds.dropna(subset = ['CREDIT RATING \n'],inplace=True)

In [115]:
# from google.colab import files
# df = bonds['CREDIT RATING \n']
# df.to_csv('dat.csv')
# files.download('dat.csv')

In [116]:
bonds['CREDIT RATING \n'].unique()

array(['CRISIL AAA STABLE', 'CRISIL AA Stable', 'CRISIL AA Negative',
       'CRISIL AA-Positive ', 'CRISIL A', 'CRISIL AAA ',
       'CRISIL A Stable', 'CRISIL AA STABLE', 'CRISIL AA+ STABLE',
       'CRISIL AAA Stable', 'CRISIL AA', 'CRISIL A-', 'CRISIL AA-',
       'CRISIL A+'], dtype=object)

In [117]:
# Filter by credit rating
# bonds= bonds[bonds['CREDIT RATING \n'].isin(['AAA', 'AA+', 'AA', 'A+', 'A','AA ' ])]

# Filter by maturity (1–5 years from now)

today = dt.datetime.today()
cutoff = today + dt.timedelta(days=5*365)
bonds['MATURITY DATE \n'] = pd.to_datetime(bonds['MATURITY DATE \n'], errors='coerce')
bonds = bonds[(bonds['MATURITY DATE \n'] >= today) & (bonds['MATURITY DATE \n'] <= cutoff)]

# Filter by volume
bonds = bonds[bonds['VOLUME \n(Shares)'] > 5]

# You can manually tag 'Issuer Type' based on Symbol or ISIN
bonds_filtered = bonds[['SYMBOL \n', 'CREDIT RATING \n', 'COUPON RATE \n', 'LTP \n', 'MATURITY DATE \n', 'VOLUME \n(Shares)','FACE VALUE \n']]
bonds_filtered


,SYMBOL \n,CREDIT RATING \n,COUPON RATE \n,LTP \n,MATURITY DATE \n,VOLUME \n(Shares),FACE VALUE \n
2,96IIFL28,CRISIL AA Stable,9.60,952.00,2028-11-03,4008,1000.0
3,734IRFC28,CRISIL AAA STABLE,7.34,"1,078.00",2028-02-19,2515,1000.0
4,96IIFL28A,CRISIL AA Negative,9.60,956.00,2028-06-24,2760,1000.0
5,871REC28,CRISIL AAA STABLE,8.71,"1,123.36",2028-09-24,2163,1000.0
9,893NHB29,CRISIL AAA STABLE,8.93,"5,570.00",2029-03-24,210,5000.0
10,1003ISFL28,CRISIL AA-Positive,10.03,990.00,2028-12-21,1181,1000.0
11,875NHAI29,CRISIL AAA STABLE,8.75,"1,146.06",2029-02-05,930,1000.0
14,1065NFL27,CRISIL A,10.65,"1,002.00",2027-03-13,1001,1000.0
16,888NHB29,CRISIL AAA STABLE,8.88,"5,600.00",2029-01-13,174,5000.0
21,863IRFC29,CRISIL AAA STABLE,8.63,"1,116.00",2029-03-26,535,1000.0


In [118]:
# Example raw ratings from your dataset
raw_ratings = [                                            # observed from the filtererd data set
    'CRISIL AAA STABLE', 'CRISIL AA Stable', 'CRISIL AA Negative',
    'CRISIL AA-Positive ', 'CRISIL A', 'CRISIL AAA ',
    'CRISIL A Stable', 'CRISIL AA STABLE', 'CRISIL AA+ STABLE',
    'CRISIL AAA Stable', 'CRISIL AA', 'CRISIL A-', 'CRISIL AA-',
    'CRISIL A+'
]

# Define a mapping function
def normalize_rating(rating):
    rating = rating.strip().upper()
    if 'AAA' in rating:
        return 'AAA'
    elif 'AA' in rating:
        return 'AA'
    elif 'A' in rating:
        return 'A'
    elif 'BBB' in rating:
        return 'BBB'
    elif 'BB' in rating:
        return 'BB'
    elif 'B' in rating:
        return 'B'
    elif 'C' in rating:
        return 'C'
    elif 'D' in rating:
        return 'D'
    else:
        return 'Other'

# Gsec-yield (for discount Rate)
gsec_yield = 7

# Function to calculate MTM using G-Sec yield as discount rate
def calculate_mtm(row, discount_rate):
    face_value = row['FACE VALUE \n']
    coupon_rate = row['COUPON RATE \n'] / 100  # convert to decimal
    maturity_date = row['MATURITY DATE \n']

    # Calculate years to maturity
    years_to_maturity = (maturity_date - dt.datetime.now()).days / 365

    # Annual coupon payment
    annual_coupon = face_value * coupon_rate

    # Calculate MTM by discounting all future cash flows
    mtm = 0
    for year in range(1, int(years_to_maturity) + 1):
        mtm += annual_coupon / ((1 + discount_rate/100) ** year)  # discount_rate in decimal

    # Add discounted principal repayment
    mtm += face_value / ((1 + discount_rate/100) ** years_to_maturity)

    return mtm

# Add MTM column using G-Sec yield as discount rate
bonds_filtered['MTM_Gsec'] = bonds_filtered.apply(lambda row: calculate_mtm(row, gsec_yield), axis=1)



<ipython-input-118-e9d481e6d33c>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonds_filtered['MTM_Gsec'] = bonds_filtered.apply(lambda row: calculate_mtm(row, gsec_yield), axis=1)


In [119]:
bonds_filtered['MAPPED RATING'] = bonds_filtered['CREDIT RATING \n'].map(normalize_rating)

<ipython-input-119-bc743143082e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bonds_filtered['MAPPED RATING'] = bonds_filtered['CREDIT RATING \n'].map(normalize_rating)


In [120]:
bonds_filtered

,SYMBOL \n,CREDIT RATING \n,COUPON RATE \n,LTP \n,MATURITY DATE \n,VOLUME \n(Shares),FACE VALUE \n,MTM_Gsec,MAPPED RATING
2,96IIFL28,CRISIL AA Stable,9.60,952.00,2028-11-03,4008,1000.0,1038.815186,AA
3,734IRFC28,CRISIL AAA STABLE,7.34,"1,078.00",2028-02-19,2515,1000.0,958.135926,AAA
4,96IIFL28A,CRISIL AA Negative,9.60,956.00,2028-06-24,2760,1000.0,1058.306334,AA
5,871REC28,CRISIL AAA STABLE,8.71,"1,123.36",2028-09-24,2163,1000.0,1021.314900,AAA
9,893NHB29,CRISIL AAA STABLE,8.93,"5,570.00",2029-03-24,210,5000.0,5004.661491,AAA
10,1003ISFL28,CRISIL AA-Positive,10.03,990.00,2028-12-21,1181,1000.0,1043.129468,AA
11,875NHAI29,CRISIL AAA STABLE,8.75,"1,146.06",2029-02-05,930,1000.0,1002.916320,AAA
14,1065NFL27,CRISIL A,10.65,"1,002.00",2027-03-13,1001,1000.0,979.145585,A
16,888NHB29,CRISIL AAA STABLE,8.88,"5,600.00",2029-01-13,174,5000.0,5048.159122,AAA
21,863IRFC29,CRISIL AAA STABLE,8.63,"1,116.00",2029-03-26,535,1000.0,992.775206,AAA


In [121]:
print(len(transition_matrix))
transition_matrix

7


,Issuer-months,AAA,AA,A,BBB,BB,B,C,D
Rating Category,,,,,,,,,
AAA,15796,98.83,1.16,0.01,0.00,0.00,0.00,0.00,0.00
AA,40980,2.26,96.10,1.54,0.04,0.01,0.00,0.00,0.05
A,78111,0.14,3.39,93.13,3.13,0.11,0.02,0.01,0.07
BBB,211375,0.00,0.04,3.20,91.49,4.61,0.11,0.03,0.52
BB,308532,0.00,0.00,0.01,4.14,88.96,3.73,0.15,3.01
B,241508,0.00,0.00,0.00,0.03,8.83,82.35,0.40,8.39
C,5330,0.00,0.00,0.00,0.00,0.72,19.87,55.21,24.20


In [122]:
bonds_filtered['MTM_Gsec'].sum()

np.float64(61901.486897138835)

In [123]:
bonds_filtered

,SYMBOL \n,CREDIT RATING \n,COUPON RATE \n,LTP \n,MATURITY DATE \n,VOLUME \n(Shares),FACE VALUE \n,MTM_Gsec,MAPPED RATING
2,96IIFL28,CRISIL AA Stable,9.60,952.00,2028-11-03,4008,1000.0,1038.815186,AA
3,734IRFC28,CRISIL AAA STABLE,7.34,"1,078.00",2028-02-19,2515,1000.0,958.135926,AAA
4,96IIFL28A,CRISIL AA Negative,9.60,956.00,2028-06-24,2760,1000.0,1058.306334,AA
5,871REC28,CRISIL AAA STABLE,8.71,"1,123.36",2028-09-24,2163,1000.0,1021.314900,AAA
9,893NHB29,CRISIL AAA STABLE,8.93,"5,570.00",2029-03-24,210,5000.0,5004.661491,AAA
10,1003ISFL28,CRISIL AA-Positive,10.03,990.00,2028-12-21,1181,1000.0,1043.129468,AA
11,875NHAI29,CRISIL AAA STABLE,8.75,"1,146.06",2029-02-05,930,1000.0,1002.916320,AAA
14,1065NFL27,CRISIL A,10.65,"1,002.00",2027-03-13,1001,1000.0,979.145585,A
16,888NHB29,CRISIL AAA STABLE,8.88,"5,600.00",2029-01-13,174,5000.0,5048.159122,AAA
21,863IRFC29,CRISIL AAA STABLE,8.63,"1,116.00",2029-03-26,535,1000.0,992.775206,AAA


In [124]:
np.random.choice([0,5,6,8,9,4])

np.int64(8)

In [125]:
transition_matrix.columns

Index(['Issuer-months', 'AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'C', 'D'], dtype='object')

In [126]:
len(transition_matrix)

7

In [129]:
transition_matrix

,Issuer-months,AAA,AA,A,BBB,BB,B,C,D
Rating Category,,,,,,,,,
AAA,15796,98.83,1.16,0.01,0.00,0.00,0.00,0.00,0.00
AA,40980,2.26,96.10,1.54,0.04,0.01,0.00,0.00,0.05
A,78111,0.14,3.39,93.13,3.13,0.11,0.02,0.01,0.07
BBB,211375,0.00,0.04,3.20,91.49,4.61,0.11,0.03,0.52
BB,308532,0.00,0.00,0.01,4.14,88.96,3.73,0.15,3.01
B,241508,0.00,0.00,0.00,0.03,8.83,82.35,0.40,8.39
C,5330,0.00,0.00,0.00,0.00,0.72,19.87,55.21,24.20


In [146]:
from tqdm import tqdm
# Example: Simplified transition matrix (rows = current rating, columns = next year's rating)


# Monte Carlo parameters
num_scenarios = 10_000  # Paper uses 20,000
recovery_rate = 0.30     # 30% recovery in default

# Simulate credit migrations for each bond
def simulate_credit_events(bond_ratings, transition_matrix, num_scenarios):
    scenarios = []
    d = {'AAA':0,'AA':1,'A':2,'BBB':3,'BB':4,'B':5,'C':6}
    # Configure tqdm with more options
    with tqdm(total=num_scenarios,
              desc="Credit Migration Simulation",
              unit="scenario",
              bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}{postfix}]") as pbar:
        for _ in range(num_scenarios):
            new_ratings = []
            for rating in bond_ratings:
                probs = transition_matrix.loc[rating].iloc[1:].tolist() # prob(---/rating)
                print(f'\n {probs}')
                print(f'size of transitioin_matrix is {len(transition_matrix.columns[1:][:])} and teh size of probs is {len(probs)}')
                new_rating = np.random.choice(transition_matrix.columns[1:][:], p=probs)
                new_ratings.append(new_rating)
            scenarios.append(new_ratings)

            pbar.update(1)  # Manually update progress bar

            # Optional: Add additional information to the progress bar
            if pbar.n % 1000 == 0:  # Update stats every 1000 scenarios
                pbar.set_postfix({
                    'current_rating': rating,
                    'scenarios': len(scenarios)
                })

    return scenarios# Example usage
bond_ratings = bonds_filtered['MAPPED RATING'].tolist()
credit_scenarios = simulate_credit_events(bond_ratings, transition_matrix, num_scenarios)

Credit Migration Simulation:   0%|          | 0/10000 [00:00<?, ?scenario/s]


 [2.26, 96.1, 1.54, 0.04, 0.01, 0.0, 0.0, 0.05]
size of transitioin_matrix is 8 and teh size of probs is 8


ValueError: probabilities do not sum to 1

In [ ]:
# Simplified example: Generate random losses based on ratings
np.random.seed(42)
n_scenarios = 1000  # Use 20,000 for real cases
n_bonds = len(bonds_filtered)

# Simulate losses (replace with CreditMetrics logic)
loss_factors = np.random.uniform(low=0.3, high=1.2, size=(n_scenarios, n_bonds))
y_scenarios = np.array(bonds_filtered['LTP \n'].values) * loss_factors  # Future values under each scenario
b = bonds_filtered['LTP \n'].values  # Benchmark (no migration)